# 1902 voter registry cleaner

In [1]:
import pandas as pd
import numpy as np
import re
import pdfminer
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## 1902 bronx
before running this code: OCR 1902 bronx.
manually inspect the data and make a new copy, 1902bronxv2.xlsx, to fix any major errors in ED/AD names

In [2]:
#xlsx file
rawdata = pd.read_excel("1902bronxv2.xlsx", header=None)

In [3]:
unwanted_words = ["The City Record", "THE CITY RECORD", "OFFICIAL JOURNAL", "SUPPLEMENT", "LIST OF REGISTERED VOTERES", "BOROUGH OF"]
col_names = ["Year", "AD", "ED", "Street", "Number", "Name"]
voters = pd.DataFrame(columns=col_names)

In [4]:
#replaces unwanted words as blank
for idx, line in rawdata.iloc[:, 0].items():
    for word in unwanted_words:
        if fuzz.partial_ratio(line.strip(), word) >= 70:
            rawdata.set_value(idx, 0, "     ")
        else:
            pass
#removes all space
td = []
for idx, line in rawdata.iloc[:, 0].items():
    if line.isspace():
        td.append(idx)

rawdata.drop(rawdata.index[td], inplace=True)
rawdata.reset_index(drop=True, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [5]:
rawdata.reset_index(drop=True, inplace=True)

In [6]:
for idx, line in rawdata.iloc[:, 0].items():
    #combine hyphenated lines
    if ((line.strip()[-1:] == "-" or line.strip()[-1:] == "--")) and rawdata.iloc[idx+1, 0][0].islower():
        newline = rawdata.iloc[idx, 0] + rawdata.iloc[idx+1, 0]
        rawdata.set_value(idx, 0, newline)
        rawdata.set_value(idx+1, 0, "    ")
#remove hyphens from names
for idx, line in rawdata.iloc[:, 0].items():
    if "-" in line:
        rawdata.set_value(idx, 0, re.sub("-", "", line))

rawdata.reset_index(drop=True, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [7]:
#removes all space
tod = []
for idx, line in rawdata.iloc[:, 0].items():
    if line.isspace():
        tod.append(idx)
rawdata.drop(rawdata.index[tod], inplace=True)
rawdata.reset_index(drop=True, inplace=True)

In [8]:
#initialize values
ad = "n/a"
ed = "n/a"
streetname = "n/a"
year = 1902
number = 0
name = "n/a"
lastname = "n/a"
firstname = "n/a"
idx = 0
minitial = ""


for line in rawdata.iloc[:, 0]:
    if line.isspace()==False:
        #all caps---either an ad, ed, maybe street/name
        capscheck = "".join(x for x in line if x.isupper())
        if len(capscheck) >=4: 
            if fuzz.partial_ratio(line, "ASSEMBLY") >= 70:
                ad = line
            elif fuzz.partial_ratio(line, "ELECTION DIST.") >= 75 or fuzz.partial_ratio(line, "ELEC. DIST.") >= 70 or fuzz.partial_ratio(line, "ELECTION DISTRICT") >= 75:
                ed = line
            else: 
                #maybe a street or name 
                lowercheck = "".join(x for x in line if x.islower())
                if len(lowercheck) <= 3: 
                    streetname = line
                else:
                    entry = line
                    number = "".join(x for x in entry if x.isdigit())
                    name = ''.join(i for i in entry if not i.isdigit())
        else:
            #for sure a name
            entry = line
            number = "".join(x for x in entry if x.isdigit())
            name = ''.join(i for i in entry if not i.isdigit())
        #new row to dataframe
        new_row = [year, ad, ed, streetname, number, name]
        voters.loc[len(voters), :] = new_row
    else:
        pass

In [9]:
#fill in empty number rows
for idx, num in voters.iloc[:, 4].items():
    if num == "":
        point = idx
        while point >= 0:
            #finds the nearest filled cell that is before it
            point -= 1
            val = voters.iloc[point, 4]
            if val is not "" and voters.iloc[idx, 3] == voters.iloc[point, 3]:
                voters.set_value(idx, "Number", val)
                break
#another one to fill in missing numbers (but going forwards)
for idx, num in voters.iloc[:, 4].items():
    if num == "":
        try:
            pt = idx
            while pt >= 0:
                #finds the nearest filled cell that is after it
                pt += 1
                val = voters.iloc[pt, 4]
                if val is not "" and voters.iloc[idx, 3] == voters.iloc[pt, 3]:
                    voters.set_value(idx, "Number", val)
                    break
        except:
            pass
#clean up any misread numbers
# for idx, number in voters.iloc[:, 4].items():
#         #number smaller than the one before it and same street
#         if float(number) < float(voters.iloc[idx-1, 4]) and (voters.iloc[idx-1, 3]) == (voters.iloc[idx, 3]):
#             #similarity between surrounding numbers
#             if fuzz.ratio(str(number), str(voters.iloc[idx-1, 4])) > fuzz.ratio(str(number), str(voters.iloc[idx+1, 4])):
#                 voters.set_value(idx, "Number", voters.iloc[idx-1, 4])
#             elif fuzz.ratio(str(number), str(voters.iloc[idx-1, 4])) < fuzz.ratio(str(number), str(voters.iloc[idx+1, 4])):
#                 voters.set_value(idx, "Number", voters.iloc[idx+1, 4])
#clean streets
for idx, street in voters.iloc[:, 3].items():
    if "." in street:
        #get rid of the periods
        removep = re.sub("[.]", "", street)
        #remove special characters
        removesc = re.sub('[^a-zA-Z.,\d\s]', '', removep)
        #remove leading space
        newstreet = removesc.strip()
        voters.set_value(idx, "Street", newstreet)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [10]:
#clean names
for idx, name in voters.iloc[:, 5].items():
    if "." in name:
        #get rid of the periods
        removep = re.sub("[.]", "", name)
        #remove special characters
        removesc = re.sub('[^a-zA-Z.,\d\s]', '', removep)
        #remove leading space
        newname = removesc.strip()
        voters.set_value(idx, "Name", newname)
#remove end commas
for idx, name in voters.iloc[:, 5].items():
    if name[-1:] == ",":
        removec = name[:-1].strip()
        #remove leading space
        voters.set_value(idx, "Name", removec)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [11]:
#lines with lone letters
delete = []
for idx, entry in voters.iloc[:, 5].items():
    if len(entry)==1:
        #most likely an initial
        voters.set_value(idx-1, "Name", voters.iloc[idx-1, 5] + entry)
        delete.append(idx)
voters.drop(voters.index[delete], inplace=True)
voters.reset_index(drop=True, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [12]:
#lone suffixes
drop = []
for idx, entry in voters.iloc[:, 5].items():
    if entry.strip() == "Rev" or entry == "Gen":
        #most likely an initial
        voters.set_value(idx-1, "Name", voters.iloc[idx-1, 5] + entry)
        drop.append(idx)
voters.drop(voters.index[drop], inplace=True)
voters.reset_index(drop=True, inplace=True)

In [13]:
#isolate suffixes
suffix = []
for idx, name in voters.iloc[:, 5].items():
    if name[-2:].strip() == "Jr" or name[-2:].strip() == "Sr" or name[-2:].strip() == "jr" or  name[-2:].strip() == "sr":
        suffix.append(name[-2:].strip())
        voters.set_value(idx, "Name", name[:-2])
    elif name[-3:].strip() == "Rev" or name[-3:].strip== "rev":
        suffix.append(name[-3:].strip())
        voters.set_value(idx, "Name", name[:-3])
    else:
        suffix.append("")
voters["Suffix"] = suffix

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [14]:
#middle initial
minitial = []
for idx, name in voters.iloc[:, 5].items():
    mi = name[-1:]
    if mi.isupper() and len(re.findall(r'\w+', name)) == 3:
        minitial.append(mi)
        cleanname = name[:-1]
        voters.set_value(idx, "Name", cleanname)
    else:
        minitial.append("")
        voters.set_value(idx, "Name", name)
voters["Middle"] = minitial
#scrub commas at end
for idx, name in voters.iloc[:, 5].items():
    if name.strip()[-1:] == ",":
        voters.set_value(idx, "Name", name.strip()[:-1])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [15]:
#reindex
voters.reset_index(drop=True, inplace=True)

#scrub letters before first capital letter
for idx, name in voters.iloc[:, 5].items():
        voters.set_value(idx, "Name", re.sub("^.*?([A-Z])", "\\1", name))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [16]:
#last name first name
lname = []
fname = []
for idx, name in voters.iloc[:, 5].items():
    #comma (x, y)
    if bool(re.match(r"(?x) ^ [\w ']+ , [\w ']+ $ ", name)):
        #everything before comma = last, everything after= first
        lname.append(name.split(',')[0].strip())
        fname.append(name.split(',')[1].strip())
    #comma at end (x, y y,)
    elif bool(re.match(r"(?x) ^ [\w ']+ , [\w ']+ ,$ ", name)):
        lname.append(name.split(',')[0].strip())
        fname.append(name.split(',')[1].strip())
    #period (x.y)
    elif bool(re.match(r"(?x) ^ [\w ']+ \. [\w ']+ $ ", name)):
        lname.append(name.split('.')[0].strip())
        fname.append(name.split('.')[1].strip())
    #space (x y)
    elif bool(re.match(r"([A-Za-z']+)+ ([A-Z][A-Za-z']+)", name)):
        grouped = re.match(r"([A-Za-z']+)+ ([A-Z][A-Za-z']+)", name).groups()
        lname.append(grouped[0].strip())
        fname.append(grouped[1].strip())
    #nospace (SmithJohn)
    elif bool(re.match(r"([A-Za-z']+)([A-Z][A-Za-z']+)", name)):
        full = re.match(r"([A-Za-z']+)([A-Z][A-Za-z']+)", name).groups()
        lname.append(full[0])
        fname.append(full[1])
    else:
        lname.append("")
        fname.append("")
#add lname fname to voters
voters["Last"] = lname
voters["First"] = fname

In [17]:
#try to fix names again:
for idx, name in voters.iloc[:, 5].items():
    if voters.iloc[idx, 6] == "" and voters.iloc[idx, 7] == "":
        new = name.split(",")
        newp = name.split(".")
        if len(new) == 2:
            voters.set_value(idx, "Last", new[0].strip())
            voters.set_value(idx, "First", new[1].strip())
        if len(newp) == 2:
            voters.set_value(idx, "Last", newp[0].strip())
            voters.set_value(idx, "First", newp[1].strip())
# lone letters and suffixes

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [18]:
#drop rows with n/a name
for idx, name in voters.iloc[:, 5].items():
    if name == "n/a":
        voters = voters.drop([idx])
voters.reset_index(drop=True, inplace=True)

In [19]:
#scrub repeats and empty fname lname
todelete = []
for idx, name in voters.iloc[:, 5].items():
    try:
        if name == voters.iloc[idx-1, 5]:
            todelete.append(idx)
    except:
        pass
voters.drop(voters.index[todelete], inplace=True)
voters.reset_index(drop=True, inplace=True)
td = []
for idx, name in voters.loc[:, "Last"].items():
    if name == "" and voters.loc[idx, "First"] == "":
        td.append(idx)
voters.drop(voters.index[td], inplace=True)

In [20]:
voters.reset_index(drop=True, inplace=True)

In [21]:
voters.to_excel("1902bronxcleaned.xlsx")

before running this code: manually inspect 1902bronxcleaned.xlsx, make a copy 1902bronxcleanedv2.xlsx making edits to major errors
## run on v2

In [16]:
voters = pd.read_excel("1902bronxcleanedv2.xlsx")

In [17]:
voters = voters.drop(voters.columns[[0]], axis=1)
voters.reset_index(drop=True, inplace=True)

In [18]:
#change all NaN's to ""
voters = voters.fillna("")

In [19]:
#fixing ED/AD

ad = 34
ed = 19
clean_ad = []
for idx, assembly in voters.iloc[:, 1].items():
    if idx == 0:
        clean_ad.append(ad)
    else:
        if assembly.upper().strip() == voters.iloc[idx-1, 1].upper().strip():
            clean_ad.append(ad)
        elif assembly.upper().strip() != voters.iloc[idx-1, 1].upper().strip():
            ad += 1
            clean_ad.append(ad)
clean_ed = []
for idx, election in voters.iloc[:, 2].items():
    if idx == 0:
        clean_ed.append(ed)
    else:
        if voters.iloc[idx, 1].upper().strip() != voters.iloc[idx-1, 1].upper().strip():
            #reset ed counter
            ed = 0
        if election.upper().strip() == voters.iloc[idx-1, 2].upper().strip():
            clean_ed.append(ed)
        elif election.upper().strip() != voters.iloc[idx-1, 2].upper().strip():
            ed += 1
            clean_ed.append(ed)
#add to df
voters["ADedit"] = clean_ad
voters["EDedit"] = clean_ed

In [20]:
voters

,Year,AD,ED,Street,Number,Name,Suffix,Middle,Last,First,ADedit,EDedit
0,1902,,iqTH ELECTION DIST.,5TH AVE,2177,"Fogarty, Edward",,J,Fogarty,Edward,34,19
1,1902,,iqTH ELECTION DIST.,5TH AVE,2177,Custer Malilon,,,Custer,Malilon,34,19
2,1902,,iqTH ELECTION DIST.,5TH AVE,277,"Kratzenberg, Frank",,,Kratzenberg,Frank,34,19
3,1902,,iqTH ELECTION DIST.,5TH AVE,277,"Hofmann, Christopher",,,Hofmann,Christopher,34,19
4,1902,,iqTH ELECTION DIST.,5TH AVE,277,"Neumann, Leopold",,,Neumann,Leopold,34,19
...,...,...,...,...,...,...,...,...,...,...,...,...
40714,1902,"PART OF SECOND ASSEMBLY DISTRICT, WESTCHESTER ...",POND ST.,TERRACE PT,1,"Rosenberger, Edward",,,Rosenberger,Edward,37,11
40715,1902,"PART OF SECOND ASSEMBLY DISTRICT, WESTCHESTER ...",POND ST.,TERRACE PT,1,"Carev, Henry",,D,Carev,Henry,37,11
40716,1902,"PART OF SECOND ASSEMBLY DISTRICT, WESTCHESTER ...",POND ST.,TERRACE PT,1,"Ludlum, Wm",,,Ludlum,Wm,37,11
40717,1902,"PART OF SECOND ASSEMBLY DISTRICT, WESTCHESTER ...",POND ST.,TERRACE PT,1,"Dixon, Jos",,C,Dixon,Jos,37,11


In [21]:
#rearrange columns 
finalvoters = voters[['Year', 'AD', 'ED', 'ADedit', "EDedit", "Street", "Number", "Name", "Last", "First", "Middle", "Suffix"]]

In [22]:
#output name
finalvoters.to_excel("processed1902bronx.xlsx")

## stats
make a copy of processed1902bronx.xlsx (v2) and fix any AD/ED counter erros

In [15]:
# load final1902manhattan in
full_list = pd.read_excel("processed1902bronx.xlsx")

In [16]:
#stats
full_list.groupby('ADedit').size()

ADedit
1      4457
2      6911
3      7374
4      6738
5      7988
6      8449
7      7780
8      5008
9      8654
10     7356
11     7083
12     5669
13     6278
14     7524
15     7662
16     7325
17     7040
18     8602
19    11515
20     8330
21    17860
22     8110
23    17362
24     7513
25     7055
26     6855
27     6323
28     7912
29     9644
30     9963
31    18055
32    10907
33     8313
34     7024
dtype: int64

In [19]:
full_list.groupby("EDedit").size()

EDedit
0         2
1     12290
2     12023
3     12008
4     11224
5     12370
6     12619
7     11756
8     11711
9     11389
10    11644
11    12120
12    10849
13    11913
14    12712
15    12145
16    11749
17    10729
18    12508
19    10292
20     9992
21     7949
22     7566
23     6408
24     4019
25     3619
26     3030
27     2145
28     2179
29     1736
30     1853
31     1550
32     1719
33     1424
34      994
35     1297
36     1506
37     1029
38     1040
39     1245
40     1074
41      755
42      757
43      853
44      511
45      336
dtype: int64

In [ ]:
full_list.groupby("Street").size()